In [ ]:
import json
import math

import pandas as pd
import numpy as np
import folium

In [ ]:
funding = pd.read_excel('data/ESF-Call-1-Successful-applications-by-Constituency-and-Council.XLS', skiprows=3)
funding.head()

In [ ]:
funding.columns

In [ ]:
#Just look at the columns we're interested in, rename them and remove null rows. Also clean Org name
ni_funding_call1_2014 = funding[['ORGANISATION NAME', 'PROJECT TITLE', 'TOTAL ELIGIBLE PROJECT COST\n', 'ESF SOUGHT\n']]
ni_funding_call1_2014 = ni_funding_call1_2014.rename(index=str, columns={'TOTAL ELIGIBLE PROJECT COST\n': 'Total Eligible Project Cost', 'ESF SOUGHT\n': 'Contribution Sought', 'ORGANISATION NAME': 'Organisation Name', 'PROJECT TITLE': 'Project Title'})
ni_funding_call1_2014 = ni_funding_call1_2014[~ni_funding_call1_2014['Organisation Name'].isnull()]
ni_funding_call1_2014['Organisation Name'] = ni_funding_call1_2014['Organisation Name'].str.replace("THE", "").str.replace("LIMITED", "").str.replace("PLC", "").str.replace("LLC", "").str.replace("LBG", "").str.replace("LTD", "").str.strip().str.upper()
ni_funding_call1_2014.head()

In [ ]:
ni_funding_call2_2014 = pd.read_excel('data/ESF-Call-2-successful-applications-by-constituency-and-council-area.xls', skiprows=2)
ni_funding_call2_2014.head()

In [ ]:
ni_funding_call2_2014.columns


In [ ]:
#Just look at the columns we're interested in and rename them. Also clean Org name

ni_funding_call2_2014 = ni_funding_call2_2014[['Organisation Name', 'Project Title', 'Investment Priority', 'Total Eligible Project Cost', 'Programme Contribution (EU and DfE) - 65%']]
ni_funding_call2_2014 = ni_funding_call2_2014.rename(index=str, columns={'Programme Contribution (EU and DfE) - 65%': 'Contribution Sought'})
ni_funding_call2_2014['Organisation Name'] = ni_funding_call2_2014['Organisation Name'].str.replace("THE", "").str.replace("LIMITED", "").str.replace("PLC", "").str.replace("LLC", "").str.replace("LBG", "").str.replace("LTD", "").str.strip().str.upper()
ni_funding_call2_2014.head()

In [ ]:
#Concatenate the data for call 1 and call 2 
ni_funding_2014 = pd.concat([ni_funding_call2_2014, ni_funding_call1_2014], sort=True)
ni_funding_2014.sort_values(axis=0, by='Organisation Name')[1:5]

In [ ]:
#manually find locations for all NI organisations
niLocations = pd.DataFrame([('ACCEPTABLE ENTERPRISES', ' BT403AW'),
            ('ACCEPTABLE ENTERPRISES LTD', ' BT403AW'),       
           ('ACCESS CENTRE LTD', 'BT152GG'),
           ('ACCESS CENTRE NI', 'BT152GG'),
           ('ACTION DEAF YOUTH', 'BT42LS'),
           ('ACTION MENTAL HEALTH', 'BT234YH'),
           ('ACTION ON HEARING LOSS', 'BT28GA'),
           ('ACTION ON HEARING LOSS (FORMERLY RNID)', 'BT28GA'),
           ('ACTIVE COMMUNITIES NETWORK LIMITED', 'BT132JF'),
           ('APPLEBY CAREERS PROJECT LTD', 'BT617AE'),
           ('ASHTON COMMUNITY TRUST', 'BT152BP'),
           ('ASSOCIATION FOR REAL CHANGE NI', 'BT126TA'),
           ('BARNARDOS', 'BT43HE'),
           ('BRYSON CHARITABLE GROUP', 'BT27FE'),
           ('CLANRYE GROUP', 'BT358SW'),                            
           ('COMPASS ADVOCACY NETWORK', 'BT536JR'),
           ('COMPASS ADVOCACY NETWORK (CAN)', 'BT536JR'),                           
           ('CUSTOMISED TRAINING SERVICES', 'BT828AS'),                           
           ('DERRY & STRABANE DISTRICT COUNCIL', 'BT487NN'),                           
           ('DERRY YOUTH & COMMUNITY WORKSHOP LIMITED', 'BT486PJ'),                           
           ('DERRY YOUTH AND COMMUNITY WORKSHOP LTD', 'BT486PJ'),
           ('DFPF LTD T/A PEOPLE 1ST', 'BT96SP'),                           
           ('DISABILITY ACTION', 'BT39ED'),                           
           ('ENTERPRISE NORTHERN IRELAND', 'BT490HE'),                           
           ('ENTERPRISE NORTHERN IRELAND LTD', 'BT490HE'),                           
           ('EXTERN GROUP', 'BT364PE'),                           
           ('EXTERN N.I.', 'BT364PE'),                            
           ('EXTERN NORTHERN IRELAND', 'BT364PE'),                            
           ('FERMANAGH & OMAGH DISTRICT COUNCIL (FODC)', 'BT797BL'),                            
           ('FERMANAGH AND OMAGH DISTRICT COUNCIL', 'BT797BL'), 
           ('FIRST STEPS WOMEN CENTRE (FSWC)', 'BT701DX'),                            
           ('FIRST STEPS WOMEN\'S CENTRE', 'BT701DX'),                            
           ('GEMS NORTHERN IRELAND LTD', 'BT27DB'),                            
           ('GREENLIGHT GATEWAY', 'BT546EZ'),                            
           ('INCLUDE YOUTH', 'BT11QA'),                            
           ('JOB DIRECTIONS LTD', 'BT828AR'),                            
           ('KILCOOLEY WOMENS CENTRE', 'BT191QS'),
           ('LIMAVADY COMMUNITY DEVELOPMENT INITIATIVE (LCDI)', 'BT490AQ'),                           
           ('MENCAP', 'BT86BT'),                          
           ('NETWORK PERSONNEL', 'BT455AJ'),                           
           ('NETWORK PERSONNEL (NP)', 'BT455AJ'),                           
           ('NETWORK PERSONNEL LTD', 'BT455AJ'),                           
           ('NETWORK PERSONNEL LTD.', 'BT455AJ'),                           
           ('NIACRO', 'BT27GS'),                                       
           ('NOW GROUP', 'BBT124GN'),
           ('NOW PROJECT LTD T/A NOW GROUP', 'BBT124GN'),                           
           ('PEOPLEPLUS (WORKS) NI LIMITED', 'BT490LP'), 
           ('Poleglass Community Association @ Sally Gardens (PCA)', 'BT170UJ'),                             
           ('RAPID', 'BT96EU'),                            
           ('RAPID LTD', 'BT96EU'),                             
           ('RCD/4RS', 'BT480LU'),                             
           ('RCD/4RS REUSE WORKSHOP', 'BT480LU'), 
           ('REED IN PARTNERSHIP', 'BT323HA'),                             
           ('RNIB', 'BT14LS'),                             
           ('RNIB NORTHERN IRELAND', 'BT14LS'),                             
           ('ROE VALLEY COMMUNITY EDUCATION FORUM LTD', 'BT499BG'),                             
           ('SHANKILL WOMEN\'S CENTRE', 'BT131FD'),                             
           ('SOMETHING SPECIAL', 'BT41NT'),   
           ('SOUTH WEST COLLEGE', 'BT808DN'),                               
           ('SOUTH WEST COLLEGE (SWC)', 'BT808DN'),                               
           ('SOUTHERN REGIONAL COLLEGE', 'BT617HF'),                               
           ('SPECIALISTERNE NI C.I.C', 'BT41AF'),                               
           ('SPRINGBOARD OPPORTUNITIES LIMITED', 'BT12GX'),                               
           ('SPRINGBOARD OPPORTUNITIES LTD', 'BT12GX'),                               
           ('START360', 'BT11PR'),                               
           ('STEPPING STONES NI', 'BT274AA'),                               
           ('THE ADVANTAGE FOUNDATION LTD', 'BT388GX'),                               
           ('THE APPLEBY TRUST', 'BT617AE'),   
           ('THE CEDAR FOUNDATION', 'BT97AS'),                              
           ('THE CONSERVATION VOLUNTEERS', 'BT60BP'),                              
           ('THE CONSERVATION VOLUNTEERS (TCV)', 'BT60BP'),                              
           ('THE ORCHARDVILLE SOCIETY', 'BT68ED'),                              
           ('THE PRINCES TRUST', 'BT125GH'),                              
           ('THE WOMEN\'S CENTRE DERRY', 'BT486BB'),                              
           ('TRAINING FOR WOMEN NETWORK LTD', 'BT41NY'),         
           ('TRIANGLE HOUSING ASSOCIATION LTD', 'BT536BD'),                              
           ('TRIAX', 'BT489LH'),                              
           ('TYRONE DONEGAL PARTNERSHIP', 'BT785LU'),                              
           ('ULSTER SUPPORTED EMPLOYMENT LIMITED', 'BT133JH'),                              
           ('UPPER SPRINGFIELD DEVELOPMENT TRUST', 'BT127FP'),                              
           ('UPPER SPRINGFIELD DEVELOPMENT TRUST (USDT)', 'BT127FP'),                              
           ('USEL', 'BT133JH'),                              
           ('WADE TRAINING LTD', 'BT621BA'),                              
           ('WOMEN IN BUSINESS NI', 'BT27AQ'),                              
           ('WOMEN\'S TEC', 'BT155EH'),                              
           ('WOMEN’S CENTRE DERRY', 'BT486BB'),                              
           ('WORKFORCE TRAINING SERVICES', 'BT127AJ'),                              
           ('YOUTH ACTION NORTHERN IRELAND', 'BT16AS'),                              
           ('YOUTHACTION NORTHERN IRELAND LTD', 'BT16AS')           
                            
                           ],    
columns=['Organisation Name', 'postcode1'])
niLocations.head()


CUSTOMISED TRAINING SERVICES has more locations. So do DISABILITY ACTION, FERMANAGH AND OMAGH DISTRICT COUNCIL, INCLUDE YOUTH, MENCAP, PEOPLEPLUS (WORKS) NI LIMITED	THE CONSERVATION VOLUNTEERS	

In [ ]:
ni_funding_2014 = pd.merge(ni_funding_2014, niLocations, how='left', left_on='Organisation Name', right_on='Organisation Name')
ni_funding_2014.shape

In [ ]:
postcodes = pd.read_csv('data/ukpostcodes.csv')

In [ ]:
postcodes['postcode'] = postcodes.postcode.str.strip().str.replace(' ', '')

In [ ]:
ni_funding_2014 =  pd.merge(ni_funding_2014, postcodes,  left_on=['postcode1'], right_on=['postcode'])

In [ ]:
def make_ni_data_geo_json(coordis_data):
    def make_feature(row):
        properties = {
            property: row[property]
            for property in ['Organisation Name', 'Project Title', 'Total Eligible Project Cost', 'Contribution Sought']
            if str(row[property]) != 'nan'
        }

        return {
            'type': 'Feature',
            'geometry': {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]
            },
            'properties': properties
        }
    features = list(coordis_data.apply(make_feature, axis=1))
    return { 'type': 'FeatureCollection', 'features': features }
with open('data/ni_data.geo.json', 'w') as file:
    json.dump(make_ni_data_geo_json(
        ni_funding_2014[~pd.isnull(ni_funding_2014['latitude'])]
    ), file, sort_keys=True)